# Introduction to NLP fundamentals

Common NLP problem is sequence to sequence (seq2seq).

In [1]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Visualizing a text dataset

In [2]:
# We want to predict whether the data is a disaster or not a disaster

In [3]:
import pandas as pd

train_df = pd.read_csv("../Datasets/nlp_getting_started/train.csv")
test_df = pd.read_csv("../Datasets/nlp_getting_started/test.csv")

In [4]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle our traininig dataframe

train_df_shuffled = train_df.sample(frac=1, random_state=42) #frac is what percentage do we want to shuffle. 1=100%
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df), len(test_df)

(7613, 3263)

In [9]:
#Let's visualize some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _, text, target = row # "_" -> to get rid of index
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n {text} \n")
    print("---\n")

Target: 1 (real disaster)
Text:
 Another Mechanical Mod Explosion: Man Injured When a 'Pharaoh' Mod Blows Up in His Hand - http://t.co/O82yVXbztv http://t.co/N5KmxuVeRg 

---

Target: 0 (not real disaster)
Text:
 Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it! 

---

Target: 1 (real disaster)
Text:
 Dramatic Video Shows Plane Landing During Violent Storm http://t.co/XRgPdlSWfD 

---

Target: 0 (not real disaster)
Text:
 Police unions retard justice &amp; drain gov $ but cops vote 4 Rs so Rs go after teachers unions instead! @DCCC @VJ44 @Lawrence @JBouie @mmfa 

---

Target: 0 (not real disaster)
Text:
 Some curls come out so pretty and some look like I just finished shooting the movie Twister 

---



### Split data into Training and Validation set

In [10]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1, #10% for validation set
                                                                            random_state = 42)

In [11]:
# Check lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [12]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting Text into Numbers

### Tokenization
Direct mapping of token (word or character) to a number

In [13]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Default parameters:

text_vectorizer = TextVectorization(max_tokens= None, # how many words in the vocabulary
                                    standardize= "lower_and_strip_punctuation",
                                    split= "whitespace",
                                    ngrams= None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens = False)

2024-09-16 13:59:22.110230: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-09-16 13:59:22.110247: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-09-16 13:59:22.110254: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-09-16 13:59:22.110266: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-16 13:59:22.110277: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
# Find the average number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
# Setup text vectorization variables

max_vocab_length = 10000
max_length = 15 # How many words of a tweet does a model see

text_vectorizer = TextVectorization(max_tokens= max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [17]:
# Fit the TextVectorizer to the training text

text_vectorizer.adapt(train_sentences)

In [18]:
# Create a sample sentence and tokenize it
sample_sentece = "There's a flood in my street!"
text_vectorizer([sample_sentece])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training set

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\n Vectorized version:")
text_vectorizer(random_sentence)

Original text:
 Senators calling for emergency housing: Boxer Feinstein back plan to move #homeless vets to VA campus http://t.co/Gm80X3vutf      

 Vectorized version:


<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([8664, 1705,   10,   73, 2433, 3141, 5657,   88,  241,    5,  769,
       2093, 2619,    5, 2623])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

In [21]:
top_5_words, bottom_5_words

(['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [22]:
# This way of using TextVectorization is a naive way. Let's try a better way

## Creating an Embedding using the Embedding Layer

In [23]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                            output_dim = 128, 
                            #  input_length = max_length # deprecated
                            )
embedding

<Embedding name=embedding, built=False>

In [26]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
    \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer(random_sentence))
sample_embed

Original text:
 In a dream you saw a way to survive and you were full of joy.    

Embedded version:


<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[ 0.00477748, -0.02715932, -0.01336914, ..., -0.00496032,
         0.03248366,  0.03247643],
       [ 0.00702805, -0.01411784, -0.03022274, ..., -0.00873114,
         0.03042832,  0.03843519],
       [-0.02806116, -0.0304951 ,  0.04378172, ..., -0.04643945,
        -0.00311183,  0.02687713],
       ...,
       [ 0.00934532, -0.03988069, -0.04573271, ..., -0.01416713,
        -0.02920241, -0.03655065],
       [-0.02896441, -0.01625061,  0.01090311, ...,  0.00289761,
         0.02082955,  0.02062347],
       [ 0.00418218, -0.019865  ,  0.03594366, ..., -0.02746618,
         0.03610716, -0.03119801]], dtype=float32)>

In [31]:
# Check out a single token's embedding

sample_embed[0], sample_embed[0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00477748, -0.02715932, -0.01336914,  0.00469532, -0.0045799 ,
         0.02297547, -0.01900574,  0.02581858,  0.03322971,  0.01056124,
        -0.03437364,  0.0263843 , -0.01509899, -0.03605275,  0.04597038,
         0.0488633 , -0.01834018, -0.04163402,  0.02444179,  0.04577247,
        -0.04050811, -0.040721  , -0.02150933,  0.0209347 ,  0.04924972,
        -0.02808818,  0.04195005, -0.04261408, -0.04910109, -0.0387992 ,
        -0.00952607,  0.03783034, -0.00296817, -0.00926174,  0.0372906 ,
        -0.03437618,  0.00993032,  0.00215488,  0.01709301,  0.03024483,
        -0.02026643,  0.01820098, -0.02256419, -0.04330328, -0.00182589,
        -0.02904705,  0.00560959, -0.02132091,  0.03897117,  0.04396892,
         0.01338145, -0.00899346,  0.03636192,  0.04714442,  0.0217077 ,
         0.02448216, -0.03891378,  0.01960423,  0.00398495, -0.01218624,
         0.03054914, -0.00051594, -0.02554263,  0.00017352,  0.0430388 ,
  

## Modelling a text dataset (running a series of experiments)

### Model 0: Getting a Baseline -> SKLearn's Naive Bayes using the TF-IDF formula.

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling Pipeline
model_0 = Pipeline([    ("tfidf", TfidfVectorizer()), # convert words to numbers using TFIDF
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [38]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of 79.27%


In [39]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [40]:
train_labels

array([0, 0, 1, ..., 1, 1, 0])